<a href="https://colab.research.google.com/github/yuvalmay30/waldo-finder/blob/main/Waldo_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-162-g2611477 torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)


Setup complete ✅ (4 CPUs, 25.5 GB RAM, 38.7/166.8 GB disk)


In [3]:
# %cd yolov5
import utils
from IPython import display
from IPython.display import clear_output
from pathlib import Path
import yaml
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob


%matplotlib inline

# Data Importing

In [4]:
%rm -r ../datasets
%rm -r ../waldo-finder

rm: cannot remove '../datasets': No such file or directory
rm: cannot remove '../waldo-finder': No such file or directory


In [5]:
%cd ..
!git clone https://github.com/yuvalmay30/waldo-finder.git  # clone
%cd yolov5/

/content
Cloning into 'waldo-finder'...
remote: Enumerating objects: 4760, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 4760 (delta 5), reused 6 (delta 1), pack-reused 4744
Receiving objects: 100% (4760/4760), 923.61 MiB | 11.60 MiB/s, done.
Resolving deltas: 100% (108/108), done.
Checking out files: 100% (3093/3093), done.
/content/yolov5


In [6]:
%cp -R ../waldo-finder/datasets ../datasets
%cp -R ../drive/MyDrive/CV\ Projects/Waldo\ Finder/last_yolov5x_200epochs_freeze.pt ../yolov5/models/

In [7]:
import shutil

def copy_dataset_yaml_to_yolo_data_directory(dataset_dir_name):
  source_file = f'../datasets/{dataset_dir_name}/{dataset_dir_name}.yaml'
  destination_file = f'./data/{dataset_dir_name}.yaml'
  
  shutil.copy(source_file, destination_file)


copy_dataset_yaml_to_yolo_data_directory('source_dataset_v2')
copy_dataset_yaml_to_yolo_data_directory('source_dataset_v2_cropped_val')
copy_dataset_yaml_to_yolo_data_directory('source_dataset_v2_tiled_5_4')

# Weights & Biases

In [7]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.8 MB 6.8 MB/s 
     |████████████████████████████████| 181 kB 94.2 MB/s 
     |████████████████████████████████| 144 kB 81.5 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Training

In [ ]:
# !python train.py --batch 16 --epochs 10 --data source_dataset_v2.yaml --weights yolov5x.pt --cache --freeze 10
!python train.py --batch 16 --epochs 10 --data source_dataset_v2.yaml --weights yolov5s.pt --cache

wandb: Currently logged in as: yuvalmay30 (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=, data=source_dataset_v2.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/ultralytics/yolov5
   c65f990..e062169  pre-commit-ci-update-config -> origin/pre-comm

# Validation

In [8]:
!python val.py --weights models/last_yolov5x_200epochs_freeze.pt --data source_dataset_v2_cropped_val.yaml --half --conf-thres 0.001 --iou 0.1 --img 2048 --single-cls
# !python val.py --weights models/last_yolov5x_200epochs_freeze.pt --data source_dataset_v2.yaml --half --conf-thres 0.01 --img 2048
# !python detect.py --weights ./models/last_yolov5x_200epochs_freeze.pt --source ../datasets/source_dataset_v2_cropped_val/valid/images --conf-thres 0.01 --iou 0.45 --img 2048

val: data=/content/yolov5/data/source_dataset_v2_cropped_val.yaml, weights=['models/last_yolov5x_200epochs_freeze.pt'], batch_size=32, imgsz=2048, conf_thres=0.001, iou_thres=0.1, task=val, device=, workers=8, single_cls=True, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-162-g2611477 torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 444 layers, 86173414 parameters, 0 gradients
100% 755k/755k [00:00<00:00, 14.4MB/s]
val: Scanning '/content/datasets/source_dataset_v2_cropped_val/valid/labels' images and labels...20 found, 0 missing, 0 empty, 0 corrupt: 100% 20/20 [00:00<00:00, 614.59it/s]
val: New cache created: /content/datasets/source_dataset_v2_cropped_val/valid/labels.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 1/1 [00:14<00:00, 14.33s/it]
                 

## Visualize

In [9]:
model = torch.hub.load('./', 'custom', source='local', path='./models/last_yolov5x_200epochs_freeze.pt', force_reload=True)
model.conf = 0.001

requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per /content/yolov5/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 v6.1-162-g2611477 torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 444 layers, 86173414 parameters, 0 gradients
Adding AutoShape... 


In [10]:
%rm -r ./runs/comparison/
%mkdir runs
%mkdir runs/comparison

rm: cannot remove './runs/comparison/': No such file or directory
mkdir: cannot create directory ‘runs’: File exists


In [17]:
from typing import Tuple
import cv2
import os
import math
import pdb
import numpy as np
import pandas as pd
from google.colab.patches import cv2_imshow


def sort_predictions_by_confidence(predictions: pd.DataFrame):
  return predictions.sort_values(by=['confidence'], ascending=False)


def convert_predictions_values_to_int(predictions):
  return predictions.astype({'xmin': 'int', 'ymin': 'int', 'xmax': 'int', 'ymax': 'int'})


def get_top_predictions_from_model_results(results, top=5):
  predictions = results.pandas().xyxy[0]

  sorted_predictions = sort_predictions_by_confidence(predictions)
  top_5_predictions = sorted_predictions[:top]
  top_5_predictions = top_5_predictions[['xmin', 'ymin', 'xmax', 'ymax', 'confidence']]

  top_5_predictions = convert_predictions_values_to_int(top_5_predictions)
  top_5_predictions = top_5_predictions.to_numpy()
  return top_5_predictions


def draw_prediction_bbox(image, top_left_coordinates: Tuple, bottom_right_coordinates: Tuple):
  red_color = (0, 0, 255)
  thickness = 6
  
  cv2.rectangle(image, top_left_coordinates, bottom_right_coordinates, red_color, thickness)
  return image


def draw_text_on_image(image, confidence, location):
  text = 'Waldo ' + str(confidence)
  font = cv2.FONT_HERSHEY_SIMPLEX
  font_scale = 0.9
  red_color = (0, 0, 255)
  thickness = 2
  
  cv2.putText(image, text, location, font, font_scale, red_color, thickness)
  return image


def draw_predictions_on_image(image, predictions):
  for prediction in predictions:
    x1, y1, x2, y2, confidence = prediction
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

    image = draw_prediction_bbox(image, (x1, y1), (x2, y2))
    image = draw_text_on_image(image, confidence, (x1, y1))
  
  return image


def load_labels(labels_path: str, dir_entry: os.DirEntry):
  filename = dir_entry.name
  labels_filename = filename[:-3] + 'txt'
  labels_path = os.path.join(labels_path, labels_filename)

  with open(labels_path) as labels_file:
    labels = labels_file.readlines()

  return labels


def draw_label_bbox(image, top_left_coordinates: Tuple, bottom_right_coordinates: Tuple):
  green_color = (0, 255, 0)
  thickness = 4
  
  cv2.rectangle(image, top_left_coordinates, bottom_right_coordinates, green_color, thickness)
  return image


def extract_bbox_from_label(label, img_width, img_height): 
  w, h = img_width, img_height

  data = label.split()
  data = [float(element) for element in data]
  data = np.array(data)

  bbox = data[1:]
  x, y, w, h = bbox * (w, h, w, h)
  x1, x2 = int(x)-int(w/2), int(x)+int(w/2)
  y1, y2 = int(y)-int(h/2), int(y)+int(h/2)

  return x1, y1, x2, y2


def draw_labels_on_image(image, labels):
  h, w = image.shape[:2]

  for label in labels:
    x1, y1, x2, y2 = extract_bbox_from_label(label, w, h)
    image = draw_label_bbox(image, (x1, y1), (x2, y2))

  return image


def get_image_from_model_results(results):
  img = results.imgs[0]
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img 


def draw_labels_and_top_predictions(images_path, labels_path):
  for dir_entry in os.scandir(images_path):
    results = model(dir_entry.path, size=2048)
    top_predictions = get_top_predictions_from_model_results(results)

    img = get_image_from_model_results(results)
    img_with_drawn_predictions = draw_predictions_on_image(img, top_predictions)

    labels = load_labels(labels_path, dir_entry)
    img_with_drawn_predictions_and_labels = draw_labels_on_image(img_with_drawn_predictions, labels)
    
    filename = dir_entry.name
    cv2.imwrite('./runs/comparison/' + filename, img_with_drawn_predictions_and_labels)
  
  print('Results saved to: /runs/comparison/')


images_path = '../datasets/source_dataset_v2_cropped_val/valid/images'
labels_path = '../datasets/source_dataset_v2_cropped_val/valid/labels'

draw_labels_and_top_predictions(images_path, labels_path)


Results saved to: /runs/comparison/
